# Multi-agent Collaboration for Financial Analysis

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [5]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPER_API_KEY = userdata.get('SERPER_API_KEY')



# CrewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [7]:
from langchain_openai import ChatOpenAI

# Creating Agents

In [8]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4-turbo')
)

In [9]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4-turbo')
)

In [10]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4-turbo')
)

In [11]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4-turbo')
)

# Creating Tasks

In [12]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [13]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [14]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [15]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

# Creating The Crew

1. The **Process** class helps to delegate the workflow to the Agents (kind of like a Manager at work)

2. In the example below, I'll run this hierarchically.

3. **manager_llm** lets you choose the "manager" LLM you want to use.

In [17]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI


financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(openai_api_key = OPENAI_API_KEY, model="gpt-4-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

# Running The Crew

In [18]:

financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [19]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze the market data for AAPL, we need to deploy advanced statistical and machine learning techniques. This involves gathering and preprocessing the data, applying relevant models, and interpreting the results to identify significant trends and predict market movements. Given the complexity of this task, I will delegate the initial data analysis and modeling part to the Data Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The task involves collecting real-time and historical market data for AAPL. You need to prepr

In [20]:
from IPython.display import Markdown
Markdown(result)

After thorough analysis of the Momentum and Swing trading strategies for AAPL, we have identified various risks and considerations based on the effectiveness of the MACD, RSI, and candlestick patterns as technical indicators. Each of these tools has its strengths and limitations, which must be carefully considered in different market conditions.

1. **Momentum Trading Strategy using MACD and RSI:**
   - **MACD** is effective in identifying trend changes and momentum but may produce false signals during range-bound or sideways market conditions. Therefore, it is crucial to confirm MACD signals with other indicators or trend analysis.
   - **RSI** helps identify overbought or oversold conditions which can signal potential reversals. However, in strong trending markets, RSI levels may remain in overbought or oversold zones for an extended period, potentially leading to premature or late entries and exits.

2. **Swing Trading Strategy using Candlestick Patterns and Volume Analysis:**
   - **Candlestick Patterns** provide insights into market sentiment and possible price reversals. However, their effectiveness can vary significantly with market volatility and the trader's experience in pattern recognition. It is advisable to use these patterns in conjunction with other technical analysis tools to increase reliability.
   - **Volume Analysis** plays a critical role in confirming the strength of price movements. A price move with relatively high volume is more likely to be sustainable. In contrast, a price move with low volume might indicate a lack of conviction among traders.

3. **General Day Trading Risks:**
   - Day trading involves substantial risk, especially because it requires making quick decisions in a short time frame. The risk of significant financial losses is elevated due to the high volatility and leverage often used in day trading.
   - Psychological factors, such as fear and greed, can also play a significant role in day trading, potentially leading traders to make impulsive decisions that deviate from a planned strategy.

**Risk Mitigation Strategies:**
- **Use of Stop-Loss Orders:** To manage risks effectively, traders should use stop-loss orders to limit potential losses.
- **Diversification:** Although AAPL is a single stock, traders can diversify trading strategies and time frames to spread risk.
- **Continuous Education:** Traders must stay informed about market conditions and continuously educate themselves about new and existing trading strategies.
- **Adherence to Strategy:** It is crucial for traders to stick to their trading plan and avoid impulsive decisions driven by short-term market movements.

In conclusion, while the identified trading strategies for AAPL using MACD, RSI, candlestick patterns, and volume analysis have potential, they carry risks that require careful management through strategic planning, risk management tools, and continuous learning. The firm's moderate risk tolerance should be carefully balanced with the aggressive nature of day trading strategies to maintain a stable trading approach.

---

## 1. The Role of the Process Class in CrewAI

In CrewAI, the **Process** class is responsible for managing how tasks are executed by the crew of agents. It defines the overall workflow and collaboration patterns among agents. Essentially, the Process class abstracts away the complexities of coordinating multiple tasks and ensures that tasks are executed in an orderly and predictable manner.

There are several key aspects to the Process class:

- **Workflow Management:** It controls the order in which tasks are executed (for example, sequentially or in a hierarchical manner).

- **Task Delegation:** In certain process modes (e.g., hierarchical), it enables the delegation of tasks through a manager agent.

- **Interaction Coordination:** The process layer ensures that agents interact appropriately—whether they are working independently (as in sequential processes) or under supervision (as in hierarchical processes).

---

## 2. Types of Processes in CrewAI

CrewAI supports different execution modes via the Process class. The two primary types are:

### A. Sequential Process

- **Description:**  

  In a sequential process, tasks are executed one after the other in the order they are defined. There is no special coordination or delegation—each agent performs its assigned task, and the output of one task can feed directly into the next.
  
- **Usage Scenario:**  

  This is ideal for linear workflows where the tasks have dependencies (e.g., data must be gathered before analysis) and where no additional oversight is required.

- **Example Code:**

  ```python
  from crewai import Crew, Agent, Task, Process
  
  # Define some sample agents and tasks (using placeholders)
  agent1 = Agent(role="Data Collector", goal="Collect data", backstory="Experienced in data collection", verbose=True)
  agent2 = Agent(role="Data Analyst", goal="Analyze data", backstory="Expert in data analysis", verbose=True)
  
  task1 = Task(description="Collect customer data", expected_output="Raw customer data", agent=agent1)
  task2 = Task(description="Analyze the collected data", expected_output="Analytical report", agent=agent2)
  
  # Initialize the crew with a sequential process
  crew = Crew(
      agents=[agent1, agent2],
      tasks=[task1, task2],
      process=Process.sequential,
      verbose=True
  )
  
  # Kickoff the crew (inputs would be provided as needed)
  final_output = crew.kickoff(inputs={"customer": "John Doe"})
  print(final_output)
  ```

  *Explanation:*  
  In the above code, tasks are executed one after the other. There is no manager agent involved—each agent works independently on its assigned task.

### B. Hierarchical Process

- **Description:**  
  A hierarchical process is designed for more complex workflows where tasks benefit from additional coordination. In this mode, a **manager agent** is used to oversee the execution of tasks. The manager agent can:
  
  - **Delegate Tasks:** Decide which agent should perform which task based on roles and current context.

  - **Validate Outputs:** Check if the results produced by agents meet the necessary criteria.

  - **Reorder or Reassign Tasks:** Adapt the workflow on the fly to optimize performance.
  
- **Key Parameters:**  
  - **manager_llm:**

    This parameter specifies the language model (LLM) to be used for generating or configuring the manager agent. When you set a manager_llm, CrewAI automatically creates a manager agent that uses this LLM for planning and coordination.
  
  - **manager_agent:**  

    Alternatively, if you have a custom manager agent, you can pass it directly using the manager_agent parameter. This allows you to override the default manager generated by manager_llm.


- **Usage Scenario:**  
  Hierarchical processes are ideal when tasks require oversight or when multiple agents need to coordinate closely. This is common in applications with complex decision-making requirements, such as customer support systems that require quality assurance.

- **Example Code (Using manager_llm):**

  ```python
  from crewai import Crew, Agent, Task, Process
  from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
  import os
  
  # Define sample agents
  agent1 = Agent(role="Customer Data Collector", goal="Retrieve customer data", backstory="Specialist in data gathering", verbose=True)
  agent2 = Agent(role="Support Representative", goal="Provide excellent support", backstory="Friendly and knowledgeable", verbose=True)
  
  # Define sample tasks
  task1 = Task(description="Retrieve customer details", expected_output="Customer data", agent=agent1)
  task2 = Task(description="Draft a support response", expected_output="Support response text", agent=agent2)
  
  # Set up the manager language model using an embedding function (or any supported LLM)
  manager_llm = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-3-small")
  
  # Initialize the crew with a hierarchical process
  crew = Crew(
      agents=[agent1, agent2],
      tasks=[task1, task2],
      process=Process.hierarchical,  # Use hierarchical process mode
      manager_llm=manager_llm,         # Provide the LLM for the manager agent
      verbose=True
  )
  
  # Kick off the crew with the hierarchical process in place
  final_output = crew.kickoff(inputs={"customer": "John Doe"})
  print(final_output)
  ```

  *Explanation:*

  - **manager_llm Parameter:** Here, we pass an instance of an LLM (created using OpenAIEmbeddingFunction) to generate the manager agent automatically.

  - **Hierarchical Coordination:** In this mode, a manager agent is created behind the scenes. This agent oversees task delegation and validation, ensuring that the support response (task2) is aligned with the customer data (task1).

- **Example Code (Using a Custom Manager Agent):**

  ```python
  from crewai import Crew, Agent, Task, Process
  
  # Define a custom manager agent
  custom_manager = Agent(
      role="Custom Manager Agent",
      goal="Coordinate tasks efficiently",
      backstory="An experienced manager who oversees the workflow and validates results",
      verbose=True
  )
  
  # Define sample agents and tasks as before
  agent1 = Agent(role="Data Collector", goal="Collect data", backstory="Expert in data gathering", verbose=True)
  agent2 = Agent(role="Support Specialist", goal="Handle support queries", backstory="Customer-focused", verbose=True)
  
  task1 = Task(description="Collect data for customer inquiry", expected_output="Collected data", agent=agent1)
  task2 = Task(description="Provide support based on collected data", expected_output="Support response", agent=agent2)
  
  # Initialize the crew with a hierarchical process using a custom manager agent
  crew = Crew(
      agents=[agent1, agent2],
      tasks=[task1, task2],
      process=Process.hierarchical,
      manager_agent=custom_manager,  # Use our custom manager agent
      verbose=True
  )
  
  # Run the crew
  final_output = crew.kickoff(inputs={"customer": "Alice Smith"})
  print(final_output)
  ```

  *Explanation:*  
  - **manager_agent Parameter:** In this example, rather than providing an LLM to automatically generate a manager, we directly supply a custom manager agent. This gives you full control over the manager’s behavior.

  - **Custom Oversight:** The custom manager is now responsible for task coordination, which may include additional logic or validations that you’ve implemented in its configuration.

---

## 5. Key Considerations and Benefits

- **Flexibility:** The Process class offers flexibility by supporting different modes of execution. For simple linear workflows, a sequential process is ideal; for more complex scenarios, the hierarchical process—with a manager agent—provides enhanced control and coordination.

- **Control via Manager Parameters:**  
  - **manager_llm** allows you to plug in a powerful LLM to automatically create a capable manager agent.

  - **manager_agent** gives you the option to provide your own manager with customized behavior.

- **Improved Task Coordination:** In hierarchical processes, the manager agent can reassign tasks, validate results, and ensure that the workflow adapts dynamically to changes in context or input data.

- **Consistent Workflow:** Regardless of the process type, using the Process class ensures that the crew operates in a predictable and structured manner, which is critical for building production-grade AI systems.

---

## Conclusion

The **Process** class in CrewAI is a crucial component for orchestrating how agents execute tasks. By supporting both sequential and hierarchical processes, it provides a range of options:

- **Sequential Process:** Ideal for linear, dependency-based workflows.
- **Hierarchical Process:** Uses either a generated manager agent (via manager_llm) or a custom manager agent (via manager_agent) to coordinate tasks, validate outputs, and delegate responsibilities effectively.


### 1. Process Class & Types
The `Process` class determines how agents collaborate. CrewAI offers 3 main process types:

#### a. Sequential Process
Tasks executed in strict order:
```python
from crewai import Crew, Agent, Task, Process

researcher = Agent(role="Researcher", goal="Find new insights")
writer = Agent(role="Writer", goal="Create engaging content")

task1 = Task(description="Investigate AI trends", agent=researcher)
task2 = Task(description="Write blog post", agent=writer)

crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    process=Process.SEQUENTIAL  # Default process
)
result = crew.kickoff()
```

#### b. Hierarchical Process
Manager agent delegates tasks using manager_llm:
```python
from crewai import Crew, Agent, Task, Process
from langchain_openai import ChatOpenAI

manager_llm = ChatOpenAI(model="gpt-4", temperature=0.3)

manager = Agent(
    role="Project Manager",
    goal="Optimize task distribution",
    backstory="Expert in resource allocation",
    llm=manager_llm
)

crew = Crew(
    agents=[manager, researcher, writer],
    tasks=[task1, task2],
    process=Process.HIERARCHICAL,
    manager_llm=manager_llm,
    verbose=2
)
```

#### c. Autonomous Collaboration
Agents self-organize through negotiation:
```python
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[task1, task2, task3],
    process=Process.AUTONOMOUS,
    memory=True  # Required for agent interaction
)
```

### 2. manager_llm Configuration
The manager-specific language model for hierarchical processes:
```python
from langchain_anthropic import ChatAnthropic

manager_llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    temperature=0.1,
    max_tokens=2000
)

crew = Crew(
    # ... other config
    process=Process.HIERARCHICAL,
    manager_llm=manager_llm
)
```

### 3. ManagerAgent Class
A specialized agent for hierarchical management:
```python
class ManagerAgent(Agent):
    def __init__(self, **kwargs):
        super().__init__(
            role="Project Manager",
            goal="Efficient task delegation",
            backstory="Seasoned tech lead with MBA",
            **kwargs
        )

    def prioritize_tasks(self, tasks):
        # Custom prioritization logic
        return sorted(tasks, key=lambda x: x.priority, reverse=True)

manager = ManagerAgent(llm=manager_llm)
```

### 4. Process Comparison Table
| Feature              | Sequential | Hierarchical | Autonomous |
|----------------------|------------|--------------|------------|
| Control Flow         | Linear     | Top-down     | Emergent   |
| Decision Making      | Predefined | Manager LLM  | Agent LLMs |
| Task Assignment      | Fixed      | Dynamic      | Negotiated |
| Best For             | Simple workflows | Complex projects | Creative tasks |
| Memory Requirement   | Optional   | Optional     | Mandatory  |

### 5. Advanced Process Configuration
Custom hierarchical process with multiple managers:
```python
from crewai.process import Process

class MultiTeamProcess(Process):
    def __init__(self):
        super().__init__("multi-team")
        
    def execute(self, crew):
        tech_manager = next(a for a in crew.agents if a.role == "Tech Lead")
        content_manager = next(a for a in crew.agents if a.role == "Content Director")
        
        tech_tasks = [t for t in crew.tasks if t.category == "technical"]
        content_tasks = [t for t in crew.tasks if t.category == "content"]
        
        tech_manager.assign_tasks(tech_tasks)
        content_manager.assign_tasks(content_tasks)
        
        return self._run_tasks(crew)

crew = Crew(
    agents=[tech_manager, content_manager, dev, writer],
    tasks=[...],
    process=MultiTeamProcess()
)
```

### 6. Process-Specific Task Attributes
```python
technical_task = Task(
    description="Implement authentication system",
    agent=developer,
    priority=0.9,
    expected_duration="2h",
    dependencies=[research_task],
    context=[user_stories]
)

content_task = Task(
    description="Create API documentation",
    agent=writer,
    priority=0.7,
    deadline="2024-03-15"
)
```

### Best Practices
1. Use **Sequential** for linear workflows (CI/CD pipelines)
2. Choose **Hierarchical** for complex projects with clear roles
3. Employ **Autonomous** for creative/exploratory tasks
4. Match manager_llm to coordination complexity
5. Monitor agent interactions in autonomous mode
6. Combine processes in nested crews for large projects

### Debugging Tips
```python
crew = Crew(
    # ... config
    verbose=2,  # Detailed execution logs
    memory_interval=2,  # Save state every 2 tasks
    process_timeout=300  # 5-minute process timeout
)
```

